# **setup selenium**

Untuk menangani konten dinamis seperti https://map.uniqlo.com/id/id/, kita perlu menggunakan Selenium, library Python yang bisa mengendalikan browser dan mensimulasikan interaksi pengguna.
`requests` dan `BeautifulSoup` tidak dapat menjalankan JavaScript, sehingga tidak bisa mensimulasikan klik.

In [10]:
!pip install webdriver-manager

In [11]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,683 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,725 kB]
Ign:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,944 kB]
Ign:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Ign:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:11 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:13 http://security.ubuntu.com/ubuntu jam

## Cari Nama Semua Outlet

In [22]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Setup opsi Chrome
options = Options()
options.add_argument("--headless")  # Hapus ini jika ingin melihat browsernya
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Inisialisasi WebDriver (ganti path jika perlu)
driver = webdriver.Chrome(options=options)

# Akses URL
url = "https://map.uniqlo.com/id/id/"
driver.get(url)

# ✅ Inisialisasi WebDriverWait
wait = WebDriverWait(driver, 15)

time.sleep(2)

# Tunggu tombol 'Lihat lokasi semua toko' dan klik
try:
    lihat_semua_button = wait.until(EC.element_to_be_clickable((
        By.XPATH,
        "//button[.//p[contains(text(), 'Lihat lokasi semua toko')]]"
    )))
    lihat_semua_button.click()
except Exception as e:
    print(f"Gagal klik tombol 'Lihat lokasi semua toko'. Error: {e}")

# Tunggu elemen outlet dimuat setelah klik
time.sleep(3)  # Bisa kamu ganti dengan wait.until jika ingin lebih aman

# Ambil semua elemen <li> yang memuat outlet
outlet_items = driver.find_elements(By.CLASS_NAME, "fr-ec-collection-list__item")

outlets = []
for item in outlet_items:
    try:
        # Cari elemen <h3> di dalam <li>
        h3 = item.find_element(By.TAG_NAME, "h3")
        outlet_name = h3.text.strip()
        outlets.append(outlet_name)
    except Exception as e:
        print(f"Error pada item: {e}")

# Tutup browser
driver.quit()

# Simpan ke DataFrame dan tampilkan
df_outlets = pd.DataFrame(outlets, columns=["Outlet"])
print(f"Jumlah outlet ditemukan: {len(df_outlets)}")
print(df_outlets)

Jumlah outlet ditemukan: 79
                                  Outlet
0                       UNIQLO 23 PASKAL
1              UNIQLO AEON MALL BSD CITY
2              UNIQLO AEON MALL DELTAMAS
3   UNIQLO AEON MALL JAKARTA GARDEN CITY
4           UNIQLO AEON MALL SENTUL CITY
..                                   ...
74      UNIQLO TRANS STUDIO MALL BANDUNG
75      UNIQLO TRANS STUDIO MALL CIBUBUR
76     UNIQLO TRANS STUDIO MALL MAKASSAR
77              UNIQLO TUNJUNGAN PLAZA 3
78                UNIQLO UNIMAS DISTRICT

[79 rows x 1 columns]


In [23]:
df_outlets

,Outlet
0,UNIQLO 23 PASKAL
1,UNIQLO AEON MALL BSD CITY
2,UNIQLO AEON MALL DELTAMAS
3,UNIQLO AEON MALL JAKARTA GARDEN CITY
4,UNIQLO AEON MALL SENTUL CITY
...,...
74,UNIQLO TRANS STUDIO MALL BANDUNG
75,UNIQLO TRANS STUDIO MALL CIBUBUR
76,UNIQLO TRANS STUDIO MALL MAKASSAR
77,UNIQLO TUNJUNGAN PLAZA 3


In [55]:
df_outlets.to_csv("data_outlet_uniqlo.csv", index=False)

## Cari Alamat Tiap Outlet

In [66]:
!pip install selenium pandas openpyxl

In [89]:
import json
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

# Setup opsi Chrome
options = Options()
options.add_argument("--headless")  # Hapus ini jika ingin melihat browsernya
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Inisialisasi WebDriver
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 15)

# 1. Buka halaman UNIQLO
driver.get("https://map.uniqlo.com/id/id/")
time.sleep(2)

# tes Klik tombol "Lihat lokasi semua toko"
try:
    lihat_button = wait.until(EC.element_to_be_clickable((
        By.XPATH,
        "//button[.//p[contains(text(), 'Lihat lokasi semua toko')]]"
    )))
    lihat_button.click()
    print("✅ Tombol 'Lihat lokasi semua toko' diklik.")
except Exception as e:
    print(f"❌ Gagal klik tombol 'Lihat lokasi semua toko': {e}")
    driver.quit()
    exit()

time.sleep(3)

all_data = []

# Step: Ambil trigger dropdown dan klik untuk tampilkan <ul>
trigger = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.fr-ec-dropdown__input-container")))
driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", trigger)
time.sleep(0.5)
driver.execute_script("arguments[0].click();", trigger)

time.sleep(1)

# Ambil semua elemen <li> dari <ul role="listbox">
province_items = wait.until(EC.presence_of_all_elements_located((
    By.CSS_SELECTOR, "ul[role='listbox'] > li.fr-ec-dropdown__option"
)))

print(f"🔽 Jumlah provinsi ditemukan: {len(province_items)}")

# Loop mulai dari indeks 1 (skip "Semua")
for i in range(1, len(province_items)):
    # Buka dropdown lagi (karena klik menutup)
    trigger = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.fr-ec-dropdown__input-container")))
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", trigger)
    trigger.click()
    time.sleep(1)

    # Re-ambil item karena DOM bisa reload
    province_items = wait.until(EC.presence_of_all_elements_located((
        By.CSS_SELECTOR, "ul[role='listbox'] > li.fr-ec-dropdown__option"
    )))
    item = province_items[i]
    province_name = item.text.strip()

    print(f"\n🔍 Memilih provinsi: {province_name}")
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", item)
    item.click()
    time.sleep(2)

    # Ambil semua nama outlet
    outlets = driver.find_elements(By.XPATH, "//li[contains(@class, 'fr-ec-collection-list__item')]//h3")
    print(f"   ➤ Ditemukan {len(outlets)} outlet")

    for h3 in outlets:
        all_data.append({
            "Provinsi": province_name,
            "Outlet": h3.text.strip()
        })

# Tutup dan simpan hasil
driver.quit()
df = pd.DataFrame(all_data)
print(df)

✅ Tombol 'Lihat lokasi semua toko' diklik.
🔽 Jumlah provinsi ditemukan: 35

🔍 Memilih provinsi: Aceh
   ➤ Ditemukan 0 outlet

🔍 Memilih provinsi: Sumatera Utara
   ➤ Ditemukan 2 outlet

🔍 Memilih provinsi: Sumatera Barat
   ➤ Ditemukan 0 outlet

🔍 Memilih provinsi: Riau
   ➤ Ditemukan 1 outlet

🔍 Memilih provinsi: Kepulauan Riau
   ➤ Ditemukan 1 outlet

🔍 Memilih provinsi: Jambi
   ➤ Ditemukan 1 outlet

🔍 Memilih provinsi: Bengkulu
   ➤ Ditemukan 0 outlet

🔍 Memilih provinsi: Sumatera Selatan
   ➤ Ditemukan 1 outlet

🔍 Memilih provinsi: Kepulauan Bangka Belitung
   ➤ Ditemukan 0 outlet

🔍 Memilih provinsi: Lampung
   ➤ Ditemukan 1 outlet

🔍 Memilih provinsi: Daerah Khusus Ibukota Jakarta
   ➤ Ditemukan 18 outlet

🔍 Memilih provinsi: Banten
   ➤ Ditemukan 7 outlet

🔍 Memilih provinsi: Jawa Barat
   ➤ Ditemukan 21 outlet

🔍 Memilih provinsi: Jawa Tengah
   ➤ Ditemukan 4 outlet

🔍 Memilih provinsi: Daerah Istimewa Yogyakarta
   ➤ Ditemukan 2 outlet

🔍 Memilih provinsi: Jawa Timur
   ➤ Dit

In [92]:
df.to_csv('data_outlet_nasional_uniqlo_id.csv', index=False)

In [91]:
df

,Provinsi,Outlet
0,Sumatera Utara,UNIQLO DELIPARK MALL
1,Sumatera Utara,UNIQLO SUN PLAZA
2,Riau,UNIQLO LIVING WORLD PEKANBARU
3,Kepulauan Riau,UNIQLO GRAND BATAM MALL
4,Jambi,UNIQLO JAMBI TOWN SQUARE
...,...,...
74,Kalimantan Selatan,UNIQLO PENTACITY
75,Kalimantan Timur,UNIQLO BIG MALL
76,Sulawesi Utara,UNIQLO MANADO TOWN SQUARE
77,Sulawesi Selatan,UNIQLO NIPAH MALL
